In [1]:
## load packages
from bs4 import BeautifulSoup
import requests
import re
import os
import numpy as np
import pandas as pd

# Import and process

In [3]:
class Data:
    def __init__(self, url):
        self.url = url
        ## names of items
        self.list_name = []
        ## status of items
        self.list_status = []
        ## total number of items
        self.list_count = []
        ## index of item for status
        self.list_idx = []
        ## link to confirmation
        self.list_link = []
    
    def get_soup(self):
        page = requests.get(self.url)
        self.soup = BeautifulSoup(page.content, 'html.parser')
    
    def get_tags(self):
        li = self.soup.findAll('li')
        self.list_total = [i.text.strip() for i in li if i.find('img', class_='thumbborder') is not None]
        self.list_href = [i.findAll('a', href=True) for i in li if i.find('img', class_='thumbborder') is not None]
        
    def extract_info(self):
        for idx, entry in enumerate(self.list_total):
            ## take left of ':', remove leading/trailing space, split on first ' '
            temp1 = entry.split(':')[0].strip().split(' ', 1)

            ## turn string at front into int
            try:
                self.list_count.append(int(temp1[0]))
            ## some have extra char, so take 1st str if so
            except:
                self.list_count.append(int(temp1[0][0]))

            ## add item name
            self.list_name.append(temp1[1])

            ## take right of ':'
            temp2 = entry.split(':')[1].strip()
            ## take data between '()'
            temp2 = re.findall('\(.*?\)',temp2)
            ## split on last space to the right, remove '(' and ')'
            temp2 = [i.replace('(', '').replace(')', '').rsplit(' ', 1) for i in temp2]
            
            count = []
            status = []
            link = []

            for i, val in enumerate(temp2):
                ## some numbers concatenated together so find all numbers
                num = [int(s) for s in re.findall(r'\s|,|[^,\s]+', val[0]) if s.isdigit()]
                count.extend(num)
                ## get data after last space to the right
                status.extend([val[1].strip()] * len(num))
                ## add link to match the number of occurrences
                link.extend([self.list_href[idx][i]['href']] * len(num))

            self.list_idx.append(count)
            self.list_status.append(status)
            self.list_link.append(link)
            
#         def add_groups(self):
#             h3 = self.soup.findAll('h3')
#             ## find indices where text != '\n'
#             idx_h3 = [m for m, n in enumerate(h3) if n.text != '\n']
#             ## find text where text != '\n'
#             self.name_h3 = [n.text for n in h3 if n.text != '\n']

#             tag_list = [tag.name for tag in soup.find_all(True)]
#             ## find index of all h3 tags
#             idx_tag = [idx for idx, tag in enumerate(tag_list) if tag == 'h3']
#             idx_h3 = [idx_tag[i] for i in idx_h3]

#             ## find index of li tags
#             idx_li = [n for n, tag in enumerate(tag_list) if tag == 'li']

#             text = [name.text for name in soup.find_all(True)]

#             idx_ukr = [i for i in idx_h3 if 'Ukraine' in text[i]][0]
#             idx_rus = [i for i in idx_h3 if 'Russia' in text[i]][0]

#             categories = [text[i] for i in idx_h3 if i not in [idx_rus, idx_ukr]]
#             name_cat = [entry.split('(')[0].strip() for entry in categories]

In [4]:
url = 'https://www.oryxspioenkop.com/2022/02/attack-on-europe-documenting-equipment.html'
Oryx = Data(url)

Oryx.get_soup()
Oryx.get_tags()
Oryx.extract_info()

# Fix errors

In [5]:
def find_error(Data):
    list_range = []
    for j in Data.list_count:
        list_range.append([i for i in range(1, j + 1)])

    idx_wrong = []
    for idx, entry in enumerate(list_range):
        if entry != Data.list_idx[idx]:
            idx_wrong.append(idx)

    # list_end = [max(i) for i in Data.list_idx]
    # list_end2 = [len(i) for i in Datas.list_idx]
    
    for i in idx_wrong:
        print(i)
        print(Data.list_total[i])
        print(Data.list_idx[i])
        print(len(Data.list_idx[i]))
        print(Data.list_count[i])
        print()
    
    return idx_wrong

In [6]:
idx_wrong = find_error(Oryx)

81
6 152mm 2A65 Msta-B howitzer: (1, destroyed by Bayraktar TB2) (2, destroyed by Bayraktar TB2) (3, destroyed by Bayraktar TB2) (4, damaged by Bayraktar TB2) (6, damaged by Bayraktar TB2) (6, abandoned)
[1, 2, 3, 4, 6, 6]
6
6

88
2 220mm BM-27 'Uragan': (1, destroyed by Bayraktar TB2) (1, abandoned and destroyed)
[1, 1]
2
2

128
120 Ural-4320: (1 and 2, destroyed by Bayraktar TB2) (3, destroyed) (4, destroyed) (5, destroyed) (6, destroyed) (7, destroyed) (8, destroyed) (9, destroyed) (10, destroyed) (11, destroyed) (12, destroyed) (13 and 14, destroyed) (15, destroyed) (16, destroyed) (17 and 18, destroyed) (19, destroyed) (20, destroyed) (21, destroyed) (22, destroyed) (23, destroyed) (24, destroyed) (25 and 26, destroyed) (27, destroyed) (28, destroyed) (29, destroyed) (30, destroyed) (31, destroyed) (32, destroyed) (33, destroyed) (34, destroyed) (35, destroyed by Bayraktar TB2) (36, destroyed by Bayraktar TB2) (37, destroyed by Bayraktar TB2) (38, destroyed by Bayraktar TB2) (39, 

In [7]:
## fix errors
Oryx.list_idx[81][4] = 5
Oryx.list_idx[88][-1] = 2
Oryx.list_idx[128][89] = 90
Oryx.list_count[187] = 9

In [8]:
## remove 2 duplicates in entries
n = 134
temp_idx = []
temp_status = []
temp_link = []

for i, j in enumerate(Oryx.list_idx[n]):
    if j not in temp_idx:
        temp_idx.append(j)
        temp_status.append(Oryx.list_status[n][i])
        temp_link.append(Oryx.list_status[n][i])
    else:
        print(j)

Oryx.list_idx[n] = temp_idx
Oryx.list_status[n] = temp_status
Oryx.list_link[n] = temp_link

34
56


In [9]:
find_error(Oryx)

[]

# Add groups

In [10]:
h3=Oryx.soup.findAll('h3')
## find indices where text != '\n'
idx_h3 = [m for m, n in enumerate(h3) if n.text != '\n']
## find text where text != '\n'
name_h3 = [n.text for n in h3 if n.text != '\n']

tag_list = [tag.name for tag in Oryx.soup.find_all(True)]
## find index of all h3 tags
idx_tag = [idx for idx, tag in enumerate(tag_list) if tag == 'h3']
idx_h3 = [idx_tag[i] for i in idx_h3]

## find index of li tags
idx_li = [n for n, tag in enumerate(tag_list) if tag == 'li']

text = [name.text for name in Oryx.soup.find_all(True)]

idx_ukr = [i for i in idx_h3 if 'Ukraine' in text[i]][0]
idx_rus = [i for i in idx_h3 if 'Russia' in text[i]][0]

categories = [text[i] for i in idx_h3 if i not in [idx_rus, idx_ukr]]
name_cat = [entry.split('(')[0].strip() for entry in categories]

In [13]:
list_indices = []
list_country = []

for i in range(len(idx_h3) - 1):
    if idx_h3[i] not in [idx_rus, idx_ukr]:
#         print(idx_h3[i])
#         print(idx_h3[i + 1])
#         print()        
        list_indices.append([idx for idx in idx_li if idx > idx_h3[i] and idx < idx_h3[i + 1]])
        if idx_h3[i] > idx_ukr:
            list_country.append('UKR')
        else:
            list_country.append('RUS')

## for last entry
list_country.append('UKR')
    
# [idx for idx in idx_li if idx > idx_h3[1] and idx < idx_h3[2]]

idx_names = np.cumsum([len(i) for i in list_indices])

group_names = []

group_names.append(Oryx.list_name[0:idx_names[0]])

for i in range(len(idx_names) - 1):
    group_names.append(Oryx.list_name[idx_names[i]:idx_names[i + 1]])
    
# group_names.append(list_name[idx_names[-1]:-1])
last_entry = Oryx.list_name[idx_names[-1]:-1]
last_entry.append(Oryx.list_name[-1])
group_names.append(last_entry)

In [14]:
names = [[Oryx.list_name[i]] * len(j) for i, j in enumerate(Oryx.list_status)]
names = [item for sublist in names for item in sublist]
status = [item for sublist in Oryx.list_status for item in sublist]
index = [item for sublist in Oryx.list_idx for item in sublist]
link = [item for sublist in Oryx.list_link for item in sublist]

In [15]:
i = 0

list_groups = []
list_countries = []

for idx, groups in enumerate(group_names):
    for group in groups:
        list_groups.append([name_cat[idx]] * Oryx.list_count[i])
        list_countries.append([list_country[idx]] * Oryx.list_count[i])
#         print([name_cat[idx]] * Oryx.list_count[i])
#         print(group)
#         print(idx)
        i += 1
#         print(i)
#     print(group)

group = [item for sublist in list_groups for item in sublist]
country = [item for sublist in list_countries for item in sublist]

In [16]:
print(len(names))
print(len(status))
print(len(index))
print(len(group))
print(len(country))
print(len(link))

1761
1761
1761
1761
1761
1761


In [17]:
df = pd.DataFrame({'names':names, 'status':status, 'idx': index, 'group':group, 'country':country, 'link': link})

In [18]:
df[['group', 'country']].value_counts()

group                              country
Trucks, Vehicles and Jeeps         RUS        410
Tanks                              RUS        214
Infantry Fighting Vehicles         RUS        194
Armoured Fighting Vehicles         RUS        137
Trucks, Vehicles and Jeeps         UKR         86
Armoured Personnel Carriers        RUS         67
Tanks                              UKR         65
Anti-Tank Guided Missiles          RUS         49
Armoured Fighting Vehicles         UKR         49
Infantry Fighting Vehicles         UKR         42
Engineering Vehicles               RUS         41
Anti-tank Guided Missiles          UKR         41
Infantry Mobility Vehicles         RUS         40
Self-Propelled Artillery           RUS         31
Surface-To-Air Missile Systems     RUS         30
Towed Artillery                    RUS         27
Multiple Rocket Launchers          RUS         23
Infantry Mobility Vehicles         UKR         22
Man-Portable Air Defence Systems   RUS         22
Armoure

In [19]:
df.group.unique()

array(['Tanks', 'Armoured Fighting Vehicles',
       'Infantry Fighting Vehicles', 'Armoured Personnel Carriers',
       'Mine-Resistant Ambush Protected', 'Infantry Mobility Vehicles',
       'Communications Stations', 'Engineering Vehicles',
       'Anti-Tank Guided Missiles', 'Man-Portable Air Defence Systems',
       'Heavy Mortars', 'Towed Artillery', 'Self-Propelled Artillery',
       'Multiple Rocket Launchers', 'Anti-Aircraft Guns',
       'Self-Propelled Anti-Aircraft Guns',
       'Surface-To-Air Missile Systems', 'Radars',
       'Jammers And Deception Systems', 'Aircraft',
       'Unmanned Aerial Vehicles', 'Helicopters', 'Logistics Trains',
       'Trucks, Vehicles and Jeeps', 'Anti-tank Guided Missiles',
       'Naval Ships'], dtype=object)

In [21]:
## save by datemod meta tag
datemod = str(Oryx.soup.findAll('meta', itemprop="dateModified", content=True))
datemod = re.findall('"([^"]*)"', datemod)[0].replace(':', '')
print(datemod)

2022-03-14T232724Z


In [22]:
## output dataframe to CSV
# cwd = os.getcwd()
path = 'assets/' + datemod + '.csv'
df.to_csv(path, index=False)

In [23]:
print(datemod)

2022-03-14T232724Z
